# Learning how to Solve a Linear Problem using Neural Networks in PyTorch

Import basic data types

In [1]:
import pandas as pd
import numpy  as np

By building our own GMRES function, we can take it appart and play with it's insides

In [2]:
from gmres import GMRES

Pyplot should display images inline

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Work with paths...

In [4]:
from os.path import join

Import image IO libraries -- might not be needed, but I'll keep it in here or now

In [5]:
from skimage.io          import imread
import matplotlib.pyplot as     pp

Creates the validation set

In [6]:
from sklearn.model_selection import train_test_split

Evaluates the model

In [84]:
from sklearn.metrics import accuracy_score
from tqdm            import tqdm, trange

Pytorch Libraries

In [8]:
import torch
from torch.autograd import Variable
from torch.nn       import Linear, ReLU, CrossEntropyLoss, \
                           Sequential, Conv2d, MaxPool2d,  \
                           Module, Softmax, BatchNorm2d, Dropout
from torch.optim    import Adam, SGD

In [34]:
mat_to_a = lambda a    : np.squeeze(np.asarray(a))
matmul_a = lambda a, b : mat_to_a(np.dot(a, b))

In [35]:
A = np.matrix(
    [[1, 1], 
     [3, -4]]
)

In [36]:
xt = np.array([2,1])
b  = matmul_a(A, xt)
x0 = np.array([0, 0])

In [25]:
b

array([3, 2])

In [26]:
e = 0
nmax_iter = 5

In [27]:
x = GMRES(A, b, x0, e, nmax_iter)

In [28]:
x

[array([3, 2]), array([1.96153846, 1.30769231]), array([2., 1.])]

In [16]:
train_x = list()
train_b = list()
for i in range(10000):
    train_x.append(np.array([10*randn(), 10*randn()]))
    train_b.append(matmul_a(A, train_x[-1]))

In [17]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [90]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 100, 2, 100, 2

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)   # equivalent to b  -- in the context of Ax = b
y = torch.empty(N, D_out)  # equivalent to x  ----------- ~~ ------------
# HACK data into here
for i in range(N):
    y[i, :] = torch.from_numpy(matmul_a(A, x[i, :]))
    # for j in range(D_in):
    #     x[i,j] = train_b[i][j]
    #     y[i,j] = train_x[i][j]

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

n_steps = 10000
print_f = n_steps/100
with tqdm(total=n_steps, file=sys.stdout, leave=True) as pbar:
    for t in range(n_steps):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)
        if t % print_f == 0:
            # print(t, loss.item())
            loss_val = loss.item()
            pbar.set_description(f"t={t:6d}, loss={loss_val:.5f}")

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.update(1)

t=  9900, loss=0.02848: 100%|██████████| 10000/10000 [00:04<00:00, 2497.75it/s]


In [91]:
model.forward(x[1,:])

tensor([ 0.5003, -7.7463], grad_fn=<AddBackward0>)

In [92]:
y[1,:]

tensor([ 0.5044, -7.7441])